# Mini project: Fitting Data

## Introduction

In this project, you will determine the lifetime of an unstable subatomic particle known as the "muon" using real data. Thanks to the QuarkNet organization for the basis of this project.

At the Large Hadron Collider at CERN, protons are accelerated to extremly high energies and made to collide with one an other. The properties (energy, momentum, etc.) of particles produced in these collisions are measured by detectors such as the Compact Muon Solenoid (CMS).

<div align="center">
    <img src="https://github.com/jstupak/ComputationalPhysics/blob/master/Images/cms.jpg?raw=true" style="height:500px"/><br>
        <figcaption>The CMS detector at the LHC.</figcaption>
</div>

Contrary to expectations, muons (and other particles) don't always have the same mass.  If you measure and plot the the mass of many muons, you will find the distribution is not a delta function, but instead has some non-zero width.  The width $\Gamma$ of this distribution is related to the lifetime $\tau$ of the particle by the relation
\begin{equation}\label{lifetime}\tag{1}
\tau=\hbar/\Gamma,
\end{equation}  
where $\hbar$ is the reduced Plank constant. NB: the width here is defined as the "full width at half maximum," meaning the full width of the distribution at half its maximum value, as shown in the Figure below.

<div align="center">
    <img src="https://github.com/jstupak/ComputationalPhysics/blob/master//Images/zWidth.png?raw=true" style="height:300px"/><br>
    <figcaption>The mass disitribution for many Z bosons.</figcaption>
</div>

However, the CMS detector does not directly measure the mass of muons (or any other particle), but instead only the energy and momentum.  Fortunately, mass $m$ can be calculated from energy $E$ and momentum $\vec{p}$ according to
\begin{equation}
mc^2=\sqrt{E^2-(\vec{p}c)^2},
\end{equation}
where $c$ is the speed of light.  

Within high-energy physics, it is cutomary to use "natural units" where $\hbar=c=1$.  In such units, this equation reduces to

\begin{equation}\label{mass}\tag{2}
m=\sqrt{E^2-p^2}.
\end{equation}
In this case, energy, momentum, and mass can all be expressed in units of energy (GeV is used throughout this notebook).

To calculate the muon lifetime, you will read in real CMS data for a large number of reconstructed muons.  For each muon, you should calculate the mass using Equation \eqref{mass} and put the value in a histogram.  Then fit a <a href="https://en.wikipedia.org/wiki/Relativistic_Breit%E2%80%93Wigner_distribution">Breit-Wigner</a> function to the distribution to determine the muon width. Finally, use Equation \eqref{lifetime} to calculate the muon lifetime.

## Data Preparation

Begin by importing numpy and matplotlib.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

The input data is in comma separated value (csv) format. To view the data, navigate to: https://raw.githubusercontent.com/jstupak/ComputationalPhysics/master/Data/muons.csv

Use the numpy `loadtxt()` function to read in the csv file and store the data in a numpy array. Tell `loadtxt()` that the txt file is in csv format by specifying the argument `delimter=","`. Tell `loadtxt()` to skip the first line of the file, which contains the column labels "Run," "Event," "E," "px," "py," and "pz," by passing the argument `skiprows=1`.

`loadtxt()` returns a numpy array.  Use it's `shape` attribute to determine how many reconstructed muons are in the dataset.  You should have six columns and many rows, each representing a particular muon.

As a sanity check, print the data for the first 5 muons in the file.

From this 2D array, create simple 1D arrays for each of the six variables using numpy slicing and indexing:

## Inspect the data

The energy array `E` can be inspected in a few different ways. First, print the array to verify its contents is correct.

What are the dimensions of the energy array?

Draw a histogram of the muon energies.

From the above, it can be seen that the muons most frequently have energy between 5 and 10 GeV.

### Task 1

Calculate the magnitude of the momentum for all muons, followed by the mass. Using numpy, these calculations can be performed on the entire array simulataneously (each row of the array corresponds to a different reconstructed muon).  NB: due to experimental efects, $E^2-p^2$ may occassionally be negetive.  Be sure to skip such cases to avoid an error.

### Task 2

Make a histogram of the muon mass with a reasonable axis range and binning.

### Task 3

Remake the mass histogram, only showing the region 0.1049 GeV to 0.1065 GeV.

### Task 4

Perform a fit on this histogram and overlay the fit result on the histogram. The functional form expected for the mass distribution is the <a href="https://en.wikipedia.org/wiki/Relativistic_Breit%E2%80%93Wigner_distribution">relativistic Breit-Wigner</a> function.

For reference, here is an example fit using an exponential function:

```python

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Create some fake data with np.random.exponential
Npoints = 4000
scale    = 7      # f(x)=(1/scale)e^(-x/scale)
s = 7*np.random.exponential(scale, Npoints) - 1

# Normally we do this to plot:
# plt.hist(s, bins=100, normed=True, histtype='step',color='blue')

# Instead we can do the following such that we access the tuple
#      returned by plt.hist returns a tuple
#      which contains arrays of the information that is being plotted:
#      ( http://matplotlib.org/api/pyplot_api.html?highlight=hist#matplotlib.pyplot.hist
#    n = array containing the number of entries in each bin )
#    bins = array containing the bin edges (so the length is nbins +1)

(n,bins,patches) = plt.hist(s, bins=50,  range=(0,100), density=False, histtype='step',color='blue')

# to fit we an array of bin centers:
bin_centers = 0.5*(bins[1:] + bins[:-1])

# define the function to fit
def f(t, No, tau_mu, B):
    return No*np.exp(-t/tau_mu) + B

# Guess the parameters of the fit (to give the algorithm a place to start)
guess_No = 1
guess_tau_mu = 1
guess_B = 1

# scipy.optimize.curve_fit
# https://docs.scipy.org/doc/scipy-0.19.1/reference/generated/scipy.optimize.curve_fit.html
# curve_fit(f, xdata, ydata, p0=None, sigma=None, absolute_sigma=False, check_finite=True, bounds=(-inf, inf), method=None, jac=None, **kwargs)[source]¶
# p0 is the initial guess for the function parameters
popt, pcov = curve_fit(f, bin_centers, n, p0 = [guess_No, guess_tau_mu, guess_B])

# curve_fit returns popt which is an array of the fit parameters
fit_result_No     = popt[0]
fit_result_tau_mu = popt[1]
fit_result_B      = popt[2]

# Draw the fit function with parameters returned by the fit
plt.plot(bin_centers, f(bin_centers, fit_result_No, fit_result_tau_mu, fit_result_B  ), 'r')
plt.show()
```

Here is the functional form for the Breit-Wigner:

In [ ]:
# define the function used in the fit. In this case we use the relativistic Breit-Wigner function from above
def func_Breit_Wigner(x, particle_mass, width, K):
    return K / ((x**2 - particle_mass**2)**2 + (particle_mass*width)**2)

Before performing the fit, provide ballpark estimates for the fit parameters (to facilitate fit convergence). Look at your plot above and provide a reasonable guess for the muon mass and width by replacing "FIXME" with numerical values.

In [ ]:
guess_particle_mass = FIXME # the x-value of the mass peak
guess_width = FIXME # width of the peak halfway up; a.k.a. "full width at half max" or FWHM
guess_K = 9.4e-05  # a constant that affects the height of the curve, you'll need to tinker with this some

With the example above as a guide, fit the muon mass distribution with the Breit-Wigner function defined above.  Draw the histogram with the fit results overlaid.

Print the mass and lifetime of the muon.  Include units and use a reasonable number of decimal places.

Great job! By fitting the data you made a measurement of the mass and lifetime of the muon!

How do your values compare to the true values?  If there is a discrepancy, what do you think might be the cause?